In [ ]:
from graphviz import Digraph

dot = Digraph(comment='The Round Table')

dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')

dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')

dot



In [ ]:
from random import random

def randomize_propability(p, delta):
    p += random() * 2* delta - delta
    if p < 0:
        p = 0
    if p > 1:
        p = 1
    return p

In [ ]:
from collections import namedtuple
from random import random
from uuid import uuid4


_P_CLONE_DELTA = 0.1

_P_CONNECTIONS_DELTA = 0.1

_P_CONNECTIONS_CHANGE_LEVEL = 0.3

Node = namedtuple("Node", ("id", "p_clone", "p_connections", "connections"))

def build_connections(connections, p_connections, connected):
    new_connections = []
    if p_connections:
        p = p_connections[0]
        p_connections = p_connections[1:]
        for node in connections:
            if node.id not in connected and random() < p:
                new_connections.append(node)
                connected.add(node.id)
                new_connections.extend(build_connections(node.connections, p_connections, connected))
    return new_connections
    

def evolve(nodes):
    new_nodes = []
    for node in nodes:
        if random() < node.p_clone:
            p_connections = list(node.p_connections)
            if random() < _P_CONNECTIONS_CHANGE_LEVEL:
                len_change = 1 if random() > 0.5 else -1
                if len_change == -1 and len(p_connections) > 1:
                    p_connections = p_connections[:-1]
                elif len_change == 1:
                    p_connections = p_connections + [p_connections[-1]]
            p_connections = [randomize_propability(p, _P_CONNECTIONS_DELTA) for p in p_connections]
            connected = set()
            connections = build_connections([node], p_connections, connected)
            new_nodes.append(
                Node(uuid4(), randomize_propability(node.p_clone, _P_CLONE_DELTA), p_connections, connections))
    return new_nodes


In [ ]:
nodes = [Node(uuid4(), 0.5, [0.5], [])]

In [ ]:
nodes.extend(evolve(nodes))
nodes

In [ ]:
from graphviz import Digraph
from itertools import chain

def find_edges(node, visited_nodes):
    edges = []
    for c in node.connections:
        edges.append((node.id, c.id))
        if c.id not in visited_nodes:
            visited_nodes.append(c.id)
            edges.extend(find_edges(c, nodes))
    return edges

visited_nodes = set((n.id for n in nodes))
edges = chain.from_iterable([find_edges(n, visited_nodes) for n in nodes])

dot = Digraph()
for a, b in edges:
    dot.edge(str(a)[:5], str(b)[:5])
dot

In [ ]:
from collections import namedtuple
from random import random
from uuid import uuid4


_P_CLONE_DELTA = 0.1

_P_CONNECTIONS_DELTA = 0.1

_P_CONNECTIONS_CHANGE_LEVEL = 0.3

Node = namedtuple("Node", ("id", "p_clone", "p_connections", "connections"))

def build_connections(connections, p_connections, connected):
    new_connections = []
    if p_connections:
        p = p_connections[0]
        p_connections = p_connections[1:]
        for node in connections:
            if node.id not in connected and random() < p:
                new_connections.append(node)
                connected.add(node.id)
                new_connections.extend(build_connections(node.connections, p_connections, connected))
    return new_connections
    

def evolve(nodes):
    new_nodes = []
    for node in nodes:
        if random() < node.p_clone:
            p_connections = list(node.p_connections)
            if random() < _P_CONNECTIONS_CHANGE_LEVEL:
                len_change = 1 if random() > 0.5 else -1
                if len_change == -1 and len(p_connections) > 1:
                    p_connections = p_connections[:-1]
                elif len_change == 1:
                    p_connections = p_connections + [p_connections[-1]]
            p_connections = [randomize_propability(p, _P_CONNECTIONS_DELTA) for p in p_connections]
            connected = set()
            connections = build_connections([node], p_connections, connected)
            new_nodes.append(
                Node(uuid4(), randomize_propability(node.p_clone, _P_CLONE_DELTA), p_connections, connections))
    return new_nodes


In [ ]:
from collections import namedtuple
from random import random, choice, randint
from math import ceil
from uuid import uuid4


_P_CLONE_DELTA = 0.1

_P_CONNECTIONS_DELTA = 0.1

_P_CONNECTIONS_CHANGE_LEVEL = 0.3

Node = namedtuple("Node", ("id", "connections"))

def reduce_nodes(nodes, target_count):
    if target_count > 0:
        nodes_to_remove = [nodes.pop(randint(0, len(nodes)-1)).id for _ in range(len(nodes) - target_count)]
        for n in nodes:
            for r in nodes_to_remove:
                try:
                    n.connections.remove(r)
                except ValueError:
                    pass
    return nodes

def count_conntections(nodes):
    return sum(map(lambda n:len(n.connections), nodes))

def reduce_connections(nodes, target_count):
    n_connctions = count_conntections(nodes)
    if target_count > len(nodes):
        connections_to_remove = n_connctions - target_count
        while connections_to_remove > 0:
            node = choice(nodes)
            n_conn = len(node.connections)
            if n_conn > 1:
                del node.connections[randint(0, n_conn-1)]
                connections_to_remove -= 1
    return nodes

def clone(nodes):
    new_ids = {n.id: uuid4() for n in nodes}
    return [Node(new_ids[n.id], [new_ids[c] for c in n.connections]) for n in nodes]

def combine(nodes_a, nodes_b):
    nodes_b = clone(nodes_b)
    nodes_a = clone(nodes_a)

    for node_a in nodes_a:
        for node_b in nodes_b:
            node_a.connections.append(node_b.id)
            node_b.connections.append(node_a.id)

    nodes = clone(nodes_a + nodes_b)
    target_count = ceil(len(nodes_a) + len(nodes_b) / 2) + randint(-1, 1) 
    reduced_nodes = reduce_nodes(nodes, target_count) 
    n_connctions = count_conntections(reduced_nodes)
    target_count = randint(n_connctions//3, (2*n_connctions)//3)

    return reduce_connections(reduced_nodes, target_count)

def evolve(generation):
    copy_generation = list(generation)
    new_generation = []
    for index, nodes in enumerate(generation):
        copy_generation.remove(nodes)
        to_nodes = choice(copy_generation)
        new_generation.append(combine(nodes, to_nodes))
        copy_generation.append(nodes)

    return new_generation


In [ ]:
generation = [[Node(uuid4(), [])],[Node(uuid4(), [])],[Node(uuid4(), [])],[Node(uuid4(), [])],[Node(uuid4(), [])]]
generation_count = 0


In [ ]:
generation = evolve(generation)
generation_count += 1

from graphviz import Digraph

dot = Digraph()
dot.format = 'png'

for nodes in generation:
    for node in nodes:
        dot.node(str(node.id)[:6])
        for c in node.connections:
            dot.edge(str(node.id)[:6], str(c)[:6])

dot.render('test' + str(generation_count))
dot

